<a href="https://colab.research.google.com/github/RaviGoti6/FakeNewsDetection/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import itertools
import re ,string, csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords # For Removing Stop words like < the , an , is ,..etc >
n_words= stopwords.words('english') #specify english stop words only
n_words.append("rt") #append rt for stop word dictionary

from nltk.tokenize import word_tokenize # for Tokenizing the sentnces as tokens
from nltk.stem.porter import PorterStemmer # converting words to their root forms ,speed and simplicity
porter = PorterStemmer() #Create stemmer obejct

from nltk.stem import WordNetLemmatizer # also converting words to their actual root forms(noun , verb ,aobjective) ,but it slow
lemmatizer = WordNetLemmatizer() #Create lemmatizer obejct

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [45]:
from google.colab import files
uploaded = files.upload()

Saving news_train.csv to news_train.csv


In [3]:
#Read the data
test_df=pd.read_csv('news_test.csv')

#Get shape and head
test_df.shape
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [4]:
train_df = pd.read_csv('news_train.csv')

train_df.shape
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
train_df = train_df.drop(columns = ['id', 'title', 'author'])

train_df

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1
...,...,...
20795,Rapper T. I. unloaded on black celebrities who...,0
20796,When the Green Bay Packers lost to the Washing...,0
20797,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [6]:
test_df = test_df.drop(columns = ['id', 'title', 'author'])

test_df

,text
0,"PALO ALTO, Calif. — After years of scorning..."
1,Russian warships ready to strike terrorists ne...
2,Videos #NoDAPL: Native American Leaders Vow to...
3,"If at first you don’t succeed, try a different..."
4,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...
5195,Of all the dysfunctions that plague the world’...
5196,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,Good morning. (Want to get California Today by...
5198,« Previous - Next » 300 US Marines To Be Deplo...


In [7]:
train_df.isnull().sum() 

text     39
label     0
dtype: int64

In [8]:
# dropping Na values
train_df = train_df.dropna()
train_df = train_df.reset_index(drop = True)

In [9]:
test_df.isnull().sum()

text    7
dtype: int64

In [10]:
# dropping Na values
tesr_df = test_df.dropna()
test_df = test_df.reset_index(drop = True)

In [11]:
train_df

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1
...,...,...
20756,Rapper T. I. unloaded on black celebrities who...,0
20757,When the Green Bay Packers lost to the Washing...,0
20758,The Macy’s of today grew from the union of sev...,0
20759,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [12]:
test_df

,text
0,"PALO ALTO, Calif. — After years of scorning..."
1,Russian warships ready to strike terrorists ne...
2,Videos #NoDAPL: Native American Leaders Vow to...
3,"If at first you don’t succeed, try a different..."
4,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...
5195,Of all the dysfunctions that plague the world’...
5196,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,Good morning. (Want to get California Today by...
5198,« Previous - Next » 300 US Marines To Be Deplo...


In [13]:
# Define Clean Function to fix text
def Clean(text):

  # Frist converting all letters to lower case
  text= text.lower()
  
  # removing unwanted digits ,special chracters from the text
  text= ' '.join(re.sub("(@[A-Za-z0-9]+)", " ", text).split()) #tags
  text= ' '.join(re.sub("^@?(\w){1,15}$", " ", text).split())
    
  text= ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())   #Links
  text= ' '.join(re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ", text).split()) 
  text= ' '.join(re.sub(r'http\S+', '',text).split())
  
  
  text= ' '.join(re.sub(r'www\S+', '',text).split())
  text= ' '.join(re.sub("\s+", " ",text).split()) #Extrem white Space
  text= ' '.join(re.sub("[^-9A-Za-z ]", "" ,text).split()) #digits 
  text= ' '.join(re.sub('-', ' ', text).split()) 
  text= ' '.join(re.sub('_', ' ', text).split()) #underscore 
   
  
  #Display available PUNCTUATION for examples for c in string.punctuation:
  #print(f"[{c}]")
  
  # removing stopwards and numbers from STRING library
  table= str.maketrans('', '', string.punctuation+string.digits)
  text = text.translate(table)
  
  # Split Sentence as tokens words 
  tokens = word_tokenize(text)
  
  # converting words to their root forms by STEMMING THE WORDS 
  #stemmed1 = [lemmatizer.lemmatize(word) for word in tokens] #Covert words to their actual root
  stemmed2 = [porter.stem(word) for word in tokens] # Covert words to their rootbut not actual
  
  # Delete each stop words from English stop words
  #words = [w for w in stemmed1 if not w in n_words] #n_words contains English stop words
  words = [w for w in stemmed2 if not w in n_words] #n_words contains English stop words

  text  = ' '.join(words)
  
  return text

In [14]:
# apply Clean Funsction to our Text
train_df.text=[Clean(x) for x in train_df.text]

# Delete Unwanted Some Text 
train_df = train_df[train_df["text"]!='btc']

In [15]:
test_df.text=test_df.text.astype(str)

In [16]:
# apply Clean Funsction to our Text
test_df.text=[Clean(x) for x in test_df.text]
# Delete Unwanted Some Text 
test_df=test_df[test_df["text"]!='btc']

In [17]:
#Get the labels from the DataFrame
labels=train_df.label
labels.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [19]:
#Split the dataset into training and testing sets
x_train,x_test,y_train,y_test = train_test_split(train_df['text'], labels, test_size=0.2, random_state=50)

In [20]:
#Initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.7)

#Fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(x_train) 
tfidf_test = tfidf_vectorizer.transform(x_test)

In [23]:
#Initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter = 50)
pac.fit(tfidf_train,y_train)

#Predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 95.3%


In [25]:
#Build confusion matrix
confusion_matrix(y_test,y_pred, labels=[1,0])

array([[1975,   83],
       [ 112, 1983]])

In [256]:
test1 = ["We now know that Virginia Thomas, the wife of Supreme Court Justice Clarence Thomas, spent the weeks after the 2020 election cheerleading the Trump White House’s efforts to overturn President Joe Biden’s victory in that election. One detail we do not yet know, however, is what Justice Thomas knew about his wife’s communications, and whether he tried to use his office to protect her.In January, the Supreme Court permitted the US House committee investigating the January 6 attacks on the Capitol to obtain hundreds of pages of White House records that may shine a light on former President Donald Trump’s efforts to thwart the peaceful transfer of power to Biden. These records may or may not contain additional evidence linking Ginni Thomas to January 6.If Clarence Thomas had his way, the House committee and the public would never know. Thomas was the only justice to publicly dissent from the Supreme Court’s decision to let the House committee obtain these records — though he offered no explanation for why he dissented.But here’s the thing: Yes, Thomas’s vote in this case, Trump v. Thompson, may have been an underhanded effort to protect his own wife. But his vote in Trump was entirely consistent with his record in cases where his spouse does not have a personal interest.In more than three decades on the Supreme Court, Thomas has consistently voted to make it harder for many Americans to have their vote count; to erode institutions, like a free press, that are essential to democracy; and to dismantle nearly a century’s worth of democratically enacted laws on spurious constitutional grounds. Thomas’s opposition to democracy is not rooted in nepotism. It appears to be quite principled.Among other things, Thomas is the only sitting justice who voted to install a Republican president in Bush v. Gore (2000) — although three other current justices were part of Republican George W. Bush’s legal team in that case. Thomas would allow Republican administrations to deactivate the entire Voting Rights Act so long as they are in power. He would strip journalists of First Amendment rights that allow them to safely provide critical coverage of government officials. And he would invalidate a long list of laws including the federal bans on child labor and on whites-only lunch counters, based on a widely rejected reading of the constitutional provision that grants Congress most of its power over the private sector. No matter how the scandal with his wife’s texts shakes out, it’s worth remembering how the Court’s longest-serving justice would shape the world. In Clarence Thomas’s America, elections would be skewed so heavily in the Republican Party’s favor that Democrats will struggle to ever gain power. And if Democrats somehow do manage to squeak into office, Thomas would ensure that they cannot govern.Thomas v. free and fair electionsThe Supreme Court’s Republican majority, in Justice Elena Kagan’s words, “has treated no statute worse” than the Voting Rights Act.It’s an astonishing attack on liberal democracy. The Voting Rights Act was America’s first meaningful attempt since Reconstruction to ensure that Black citizens could participate equally in selecting their own leaders. And, when it was fully in effect, it was a breathtakingly effective law. Just two years after President Lyndon Johnson signed the Voting Rights Act into law, Black voter registration rates in Mississippi skyrocketed from 6.7 percent to nearly 60 percent.And yet, since its 2013 decision in Shelby County v. Holder, the Court has systematically dismantled the Voting Rights Act’s key provisions. It hamstrung the law’s “preclearance” provision, which required federal officials to screen voting laws in states with a history of racist election practices to ensure that those laws do not discriminate. It imposed such a high burden of proof on voting rights plaintiffs alleging intentional discrimination that such cases are now virtually impossible to win. And the Court has fabricated limits on the Voting Rights Act that appear nowhere in the law’s text, such as a presumption that voting restrictions that were common in 1982 are valid.Justice Thomas supported all of these efforts to weaken the Voting Rights Act, a law that arguably did more than any statute in American history to dismantle Jim Crow, But he’s also consistently urged his Court to go much further. It’s unclear whether the Voting Rights Act retains any real force after its many harrowing encounters with the Roberts Court, but Thomas would all but ensure that the law is meaningless.In the late 1960s, just a few years after the Voting Rights Act became law, the Supreme Court recognized that the law “was aimed at the subtle, as well as the obvious, state regulations which have the effect of denying citizens their right to vote because of their race.”Imagine, for example, a city where 60 percent of the population is white, and 40 percent is Black. Now imagine that the city draws gerrymandered districts which ensure that white voters will be a majority in every city council district. In such a place Black voters might nominally possess the right to vote, but any vote cast by a Black person would be meaningless if the white majority hangs together to deny power to the Black minority’s preferred candidates.To prevent these kinds of subtle attacks on the right to vote, the Supreme Court has, for more than half a century, understood the Voting Rights Act to prohibit “vote dilution” — that is, laws that diminish the power of voters of color without formally stripping them of the right to vote altogether. Concurring in the judgment in Holder v. Hall (1994), however, Thomas argued that the Court should abolish vote dilution claims, and effectively allow states to deny voting rights to certain racial groups so long as the state does it with a degree of subtlety.In Holder, a majority of the Court concluded that vote dilution claims could not be used to challenge the number of people who sit on a governing body, but only Justice Antonin Scalia joined Thomas’s opinion seeking to shut down vote dilution lawsuits altogether.“Properly understood,” Thomas claimed, the Voting Rights Act only forbids “practices that affect minority citizens’ access to the ballot.” “Districting systems and electoral mechanisms that may affect the ‘weight’ given to a ballot,” Thomas continued, “are simply beyond the purview of the Act.”Thus, a state would be free to lock voters of a particular race out of power entirely, just so long as those voters were allowed to perform the meaningless act of submitting a ballot in an election that their preferred candidate cannot possibly win.More recently, in Brnovich v. Democratic National Committee (2021), Thomas joined an opinion by Justice Neil Gorsuch which suggested that no private party is allowed to bring a lawsuit under the Voting Rights Act — only the US Justice Department could do so.As the Supreme Court explained in Allen v. State Board of Elections (1969), such an approach would severely hamper the law’s effectiveness, even if the Justice Department is committed to protecting voting rights. “The Attorney General has a limited staff,” the Court noted in Allen, “and often might be unable to uncover quickly” new state policies that target voters of color.And there’s no guarantee that the Justice Department will be led by people who care about voting rights. One result of the approach Thomas endorsed in Brnovich is that, in a Republican administration, the Voting Rights Act could cease to function altogether.Thomas was also an early proponent of the so-called “independent state legislature doctrine,” a theory that would allow state lawmakers to ignore their state constitution altogether when writing the laws governing congressional and presidential elections. In its strongest form, this doctrine would allow a state legislature to simply gift a state’s electoral votes to the Republican presidential candidate (or, in theory, to any presidential candidate), regardless of what the people of the state, the state’s governor, or the state’s supreme court has to say about it.Thomas would dismantle the freedom of the pressEven if states hold nominally free and fair elections where every vote counts equally, elections lose much of their import if voters cannot learn which candidates support their preferred policies or know what choices politicians make once elected. This is why a free press is essential to any democracy, because the right to vote means little if voters can’t determine who to vote for.And yet, Thomas called for his Court to overrule New York Times v. Sullivan (1964), the single most important decision enabling journalists to report the news without facing intimidation or sanction from government officials.In 1960, civil rights activists aligned with Martin Luther King, Jr. ran an advertisement in the New York Times, which alleged that Alabama police used brutal tactics to suppress student protests. The ad, however, contained some minor factual errors. It misidentified the song that protesters sang at a particular demonstration, for example, and it also claimed that police had arrested King seven times, when he’d in fact only been arrested four times.Pointing to these small errors, a Jim Crow police official won a $500,000 verdict against the Times in an Alabama court — close to $5 million in 2022 dollars. Had this verdict stood, it would have chilled journalism of all kinds, because it would have meant that any newspaper or other outlet that prints even very small factual mistakes could have been hit with a verdict large enough to bankrupt the outlet.The New York Times decision, however, prevented this outcome by holding that the First Amendment imposes limits on defamation lawsuits. When someone speaks about a public figure and about a matter of public concern, the Court held, they cannot be held liable for making false statements unless that statement was made “with knowledge that it was false or with reckless disregard of whether it was false or not.”Thomas argued in McKee v. Cosby (2019) that New York Times should be overruled. Indeed, Thomas’s opinion suggests that states should be free to define their own defamation law free of constitutional constraints. “The States are perfectly capable of striking an acceptable balance between encouraging robust public discourse and providing a meaningful remedy for reputational harm,” Thomas wrote.If this approach were to prevail, state officials could once again use malicious defamation lawsuits to target journalists. Suppose, for example, that I mistakenly report that “500 people attended a rally protesting Florida Gov. Ron DeSantis,” when in fact the rally was attended by only 450 people. If states can set their own defamation laws, free of constitutional constraint, then DeSantis could sue me and Vox Media for millions, endangering our ability to continue reporting on DeSantis — and potentially bankrupting Vox in the process.Thomas would make the winner of a federal election largely irrelevantThomas’s final avenue of attack on American democracy is perhaps even more subtle and insidious.Under Justice Thomas’s approach, the winner of a federal election is largely irrelevant, because the federal government would be stripped of its authority to do nearly anything that the current majority on the Court disapproves of.That’s because his views on the balance of power among the three branches of the federal government, and on the balance of power between Congress and the states, would leave the national government little more than an empty husk.To back up: Numerous federal statutes lay out broad policy objectives — such as power plants should use the best available technology to reduce emissions or health insurers shall cover vaccines that are recommended by medical experts — then delegates the task of implementing these objections to a federal agency. One advantage of this approach is that it allows the government to be dynamic. As new emissions reduction technology emerges, for example, the Environmental Protection Agency can update the relevant regulations to ensure that power plants remain state-of-the-art. Another is that it allows democratically elected lawmakers — with a diverse set of backgrounds — to set policy goals, but also leaves the difficult details of implementing these goals to officials with specialized expertise.In recent years, however, the Court’s Republican appointees have given themselves a veto power over all of these agency regulations. Relying on vague doctrines that appear nowhere in the Constitution, such as the “major questions” doctrine or “nondelegation,” the Court has claimed the power to strike down regulations that a majority of its members disapprove of.Thomas, however, would go even further. In a 2015 opinion, Thomas argued that any federal law that permits an agency to exercise “policy discretion” is unconstitutional. Thus, Congress would be forbidden from creating a modern environmental protection regime, or a dynamic regime where medical experts can quickly make new vaccines available to the public, no matter how the American people vote in congressional elections.Thomas would also strike down huge swaths of federal law governing the workplace and other private businesses.The Constitution permits Congress to “regulate commerce ... among the several states.” This provision is what allows the federal government to protect the right to unionize, to ban child labor, to set the minimum wage, to prohibit discrimination by private companies, and to regulate health insurers — among many other things.Concurring in United States v. Lopez (1995), however, Thomas endorsed the legal reasoning the Court used in Hammer v. Dagenhart (1918), an anti-canonical decision striking down federal child labor laws. And he’s restated this view in at least three other opinions since Lopez.For those who want a deep dive, I’ve written about the full implications of Thomas’s opinion in Lopez at considerable length. But the short version is that Thomas’s approach endangers much of the New Deal, the Great Society, and decades of other regulations of private businesses which now form a backbone of American society.Again, under Thomas’s approach, it is highly doubtful that the federal ban on whites-only lunch counters, which the Supreme Court held was a valid exercise of Congress’s power to regulate commerce in Katzenbach v. McClung (1964), could survive.Thomas, in other words, imagines a world where state lawmakers have broad authority to skew elections in their party’s favor. He would strip journalists of the First Amendment protections they need to do their job safely. And, if a left-of-center government somehow did emerge despite these constraints, Thomas would strip that government of most of its authority to govern.Ultimate power would rest with the Supreme Court, and its panel of unelected judges who serve for life, not with the American people. And Thomas would wield that power to turn back the clock on American law nearly an entire century.Will you support Vox’s explanatory journalism?Millions turn to Vox to understand what’s happening in the news. Our mission has never been more vital than it is in this moment: to empower through understanding. Financial contributions from our readers are a critical part of supporting our resource-intensive work and help us keep our journalism free for all."]

In [257]:
# apply Clean Funsction to our Text
test1 = [Clean(x) for x in test1]

# Delete Unwanted Some Text 
#test1 = test1[test1 != 'btc']

In [258]:
test1

['know virginia thoma wife suprem court justic clarenc thoma spent week elect cheerlead trump white hous effort overturn presid joe biden victori elect one detail yet know howev justic thoma knew hi wife commun whether tri use hi offic protect herin januari suprem court permit us hous committe investig januari attack capitol obtain hundr page white hous record may shine light former presid donald trump effort thwart peac transfer power biden record may may contain addit evid link ginni thoma januari clarenc thoma hi way hous committe public would never know thoma wa onli justic publicli dissent suprem court decis let hous committe obtain record though offer explan whi dissentedbut thing ye thomass vote thi case trump v thompson may underhand effort protect hi wife hi vote trump wa entir consist hi record case hi spous doe person interestin three decad suprem court thoma ha consist vote make harder mani american vote count erod institut like free press essenti democraci dismantl nearli 

In [259]:
#Initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.7)

#Fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(x_train) 
tfidf_test1 = tfidf_vectorizer.transform(test1)

In [260]:
#Initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter = 50)
pac.fit(tfidf_train,y_train)

#Predict on the test set and calculate accuracy
y_pred1 = pac.predict(tfidf_test1)
#score = accuracy_score(y_test,y_pred1)
#print(f'Accuracy: {round(score*100,2)}%')

In [261]:
y_pred1

array([1])

In [140]:
import requests
from bs4 import BeautifulSoup

In [248]:
url = 'https://www.vox.com/2022/3/29/22999755/supreme-court-clarence-thomas-voting-rights-democracy-elections-ginni'
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

output = ''
blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',]

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

output = output.replace('\t', '')

print(output)

Supreme Court Justice Clarence Thomas’s long fight against fair elections - Vox 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Skip to main content 
  Google Tag Manager (noscript)  
 
  End Google Tag Manager (noscript)  
 clock 
 menu 
 more-arrow 
 no 
 yes 
 
 
 
 
 
 
 
 Vox homepage 
 
 
 
 
 
 
 
 
 
 
 
 Give 
 Give 
 
 
 Newsletters 
 Newsletters 
 
 
 
 Site search 
 
 Search 
 Search 
 
 
 
 
 
 Vox main menu 
 
 
 
              Ukraine
              
               
 
 
 
              Coronavirus
              
               
 
 
 
              Recode
              
               
 
 
 
              The Goods
              
               
 
 
 
              Future Perfect
              
               
 
 
 
              The Highlight
              
               
 
 
 
              Crossword
              
               
 
 
 
              Podcasts
              
               
 
 
 
              Video
              
               
 
 
 
              Explaine

In [249]:
test1 = [output]

In [250]:
test1

['Supreme Court Justice Clarence Thomas’s long fight against fair elections - Vox \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n Skip to main content \n  Google Tag Manager (noscript)  \n \n  End Google Tag Manager (noscript)  \n clock \n menu \n more-arrow \n no \n yes \n \n \n \n \n \n \n \n Vox homepage \n \n \n \n \n \n \n \n \n \n \n \n Give \n Give \n \n \n Newsletters \n Newsletters \n \n \n \n Site search \n \n Search \n Search \n \n \n \n \n \n Vox main menu \n \n \n \n              Ukraine\n              \n               \n \n \n \n              Coronavirus\n              \n               \n \n \n \n              Recode\n              \n               \n \n \n \n              The Goods\n              \n               \n \n \n \n              Future Perfect\n              \n               \n \n \n \n              The Highlight\n              \n               \n \n \n \n              Crossword\n              \n               \n \n \n \n              Podcasts\